# Exercise Informed Search Algorithms #


In [1]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path().absolute()))

try:
    import search
finally:
    sys.path.pop(0)

In the last session, we implemented different systematic search strategies. If we want to find paths between different cities in a map, we can use additional information to guide our search. We don't rely on the 'blind' search and can implement more efficient algorithms, that consider the coordinates of the cities for example.

Implement the following algorithms and answer the same questions as you did for the systematic search algorithms.

1. Greedy Search
1. A* Algorithm
1. IDA* Search

In [2]:
from sbb import SBB

sbb = SBB()
sbb.importData('linie-mit-betriebspunkten.json')

start = 'Rotkreuz'
goal = 'Thalwil'
sbb_map = search.UndirectedGraph(sbb.createMap())
problem = search.GraphProblem(start, goal, sbb_map)

def greedy_search(problem, heuristic):
    return heuristic_search(problem, heuristic)


def a_star_search(problem, heuristic):
    return heuristic_search(problem, lambda n: heuristic(n) + n.path_cost)


def heuristic_search(problem, heuristic):
    explored = set()
    node = search.Node(problem.initial)

    while node is not None:
        candidates = {}

        if problem.goal_test(node.state):
            print("Found node:", node.state)
            print("   - Cost:", node.path_cost)
            print("   - # Visited Nodes:", len(explored))
            print("   - # Max Stored nodes:", len(explored))
            return node

        explored.add(node.state)
        candidates = [(child, heuristic(child)) for child in node.expand(problem) if child.state not in explored]

        if not candidates:
            return None

        candidates.sort(key=lambda c: c[1])
        node = candidates[0][0]

    return None


def iterative_a_star(problem, heuristic):
    def priority(n):
        return heuristic(n) + n.path_cost

    node = search.Node(problem.initial)

    frontier = search.PriorityQueue('min', priority)
    frontier.append(node)
    explored = set()

    while frontier:
        node = frontier.pop()

        if problem.goal_test(node.state):
            print("Found node:", node.state)
            print("   - Cost:", node.path_cost)
            print("   - # Visited Nodes:", len(explored))
            print("   - # Max Stored nodes:", len(explored) + len(frontier))
            return node

        explored.add(node.state)
        for child in node.expand(problem):
            in_frontier = len([n for _, n in frontier.heap if n.state == child.state]) > 0
            if not in_frontier and child.state not in explored:
                frontier.append(child)

    return None

successfully imported 2787 hubs
successfully imported 401 train lines


Hints: you can use the heap library heapq for your frontier:

`from heapq import heappush, heappop`

The following line will add the node `f` to the frontier with priority `f`:

`heappush(frontier, (f, node))`

To get the first node, use: `node = heappop(frontier)[1]`

The aerial distance between a node and the goal can be computed with the following function:

`sbb.get_distance_between(node.state, problem.goal)`
        

How do theses informed search algorithms perform on our problem? Create the following overview table for the example problem.


| Algorithm | start   | goal | cost | number of nodes visited | maximal stored nodes | complete | optimal |
|------|------|-----|-----|-----|-----|-----|-----|
| Greedy Search| Rotkreuz | Thalwil | 37 | 16 | 16 | No | No |
| A\*| Rotkreuz | Thalwil | n/a | n/a | n/a | Yes  | Yes |
| IDA\*| Rotkreuz | Thalwil | 36 | 140 | 174 | Yes  | Yes |

Note: A\* does not find a solution because it gets stuck in a local maxima.


In [3]:
heuristic = lambda n: sbb.get_distance_between(n.state, problem.goal)

In [4]:
greedy_node = greedy_search(problem, heuristic)
assert greedy_node, "Couldn't find a solution to the problem :("

Found node: Thalwil
   - Cost: 37.272000000000006
   - # Visited Nodes: 16
   - # Max Stored nodes: 16


In [5]:
a_star_node = a_star_search(problem, heuristic)
assert a_star_node, "Couldn't find a solution to the problem :("

AssertionError: Couldn't find a solution to the problem :(

In [6]:
ierative_a_star_node = iterative_a_star(problem, heuristic)
assert ierative_a_star_node, "Couldn't find a solution to the problem :("

Found node: Thalwil
   - Cost: 36.906
   - # Visited Nodes: 146
   - # Max Stored nodes: 163
